In [2]:
import sqlalchemy as sa
from sqlalchemy import create_engine, text, MetaData
import pymysql
import sqlite3
import win32com.client as win32 


import numpy as np
import pandas as pd
from datetime import datetime, timedelta,date

from openpyxl import load_workbook
from openpyxl.utils import get_column_letter

from data_logger import setup_logger
import module_query_update
import module_broker_tag_compare 
import warnings
import os
from io import StringIO
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=pd.errors.SettingWithCopyWarning)

logger = setup_logger()

In [3]:
def EmailAutomation(file_path, mail_to, mail_cc,edit_df):      ###可以cc自己检查
        absolute_path = os.path.abspath(file_path)
        filename = os.path.basename(absolute_path)
        text = f'共有{len(edit_df)}张票需要修改，其中{edit_df.loc[edit_df['SAAS_rating']==edit_df['手工调整意见']].shape[0]}张票维持原有评级,{edit_df.loc[edit_df['SAAS_rating']>edit_df['手工调整意见']].shape[0]}张票评级上调，{edit_df.loc[edit_df['SAAS_rating']<edit_df['手工调整意见']].shape[0]}张票评级下调'
        text_2 =f'获取的数据日期为{edit_df['trade_date'].unique()[0]},修改日期为{edit_df['edit_date'].unique()[0]}。'
        outlook = win32.Dispatch('outlook.application')
        mail = outlook.CreateItem(0)
        mail.To = mail_to
        mail.CC = mail_cc
        mail.Subject = f"白名单修改及{filename}"
        mail.Body = 'Email automation'
        mail.BodyFormat = 2
        mail.HTMLBody = f"""
        <html>
        <head>
        <style>
                body {{ font-family: 'Microsoft Yahei', sans-serif; font-size: 14px; color: #333333; }}
                .greeting {{ font-weight: bold; }}
                .signature {{ font-weight: bold; margin-top: 20px; display:block; }}
                .instruction, .attachment-note {{ margin-top: 10px; display:block; }}
        </style>
        </head>
        <body>
        <p class="greeting">明非姐好：</p>
        <p>本周白名单更新状况如下：<br>{text}</p>
        <p>{text_2}</p>
        <p class="instruction">本周白名单调整明细在附件中，请您查收。</p>
        <p class="attachment-note">请将附件更新至飞书备份! 谢谢!</p>
        <p class="signature">Best Wishes</p>
        </body>
        </html>
        """
        # mail.Importance = 2 
        mail.Attachments.Add(absolute_path)
        mail.Send()

        logger.info(f"邮件发送至{mail_to}, \n已抄送{mail_cc}")

def init():  
    istcok_db=module_query_update.getConnection('istock')
    rating_db=create_engine('sqlite:///result/rating_db.db')
    end_date = pd.read_sql("SELECT MAX(td) FROM model_st ms ;",istcok_db)
    end_date=end_date.loc[0][0].strftime('%Y-%m-%d')
    start_date = pd.read_sql("select max(trade_date) from saas_rating;",rating_db)
    start_date=start_date.loc[0][0]
    sql_rating = f'''SELECT MIN(td_date) AS min_td_date
    FROM (
        SELECT  DISTINCT td_date 
        FROM shsz_stock_daily_quotation
        WHERE td_date <= '{end_date}'
        ORDER BY td_date DESC
        LIMIT 5
    ) AS recent_dates;'''
    date_df=pd.read_sql(sql_rating,istcok_db)
    from_date=date_df.loc[0][0].strftime('%Y-%m-%d')
    del date_df
    query_config= {'start_date': start_date,
                'end_date': end_date,
                'istock_conn': istcok_db,
                'rating_conn' : rating_db,
                'from_date': from_date}
    return query_config

def save_edit_log(edit_df):
    conn = sqlite3.connect('result/rating_db.db')
    cursor = conn.cursor()

    # 创建表（如果表不存在）
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS edit_log (
        stock_code TEXT,
        stock_name TEXT,
        trade_date TEXT,
        SAAS_rating TEXT,
        中信建投 TEXT,
        华泰证券 TEXT,
        国泰君安 TEXT,
        tag TEXT,
        AI_comment TEXT,
        AI_rating TEXT,
        edit_rating TEXT,
        edit_date TEXT,
        PRIMARY KEY (stock_code, trade_date)
    )
    """)
    # 将DataFrame中的数据插入到edit_log表中
    for index, row in edit_df.iterrows():
        cursor.execute("""
        INSERT OR IGNORE INTO edit_log (stock_code, stock_name, trade_date, SAAS_rating, 中信建投, 华泰证券, 国泰君安, tag, AI_comment, AI_rating, edit_rating, edit_date)
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            str(row['stock_code']),  # 确保 stock_code 是字符串类型
            row['stock_name'],
            row['trade_date'],
            row['SAAS_rating'],
            row['中信建投'],
            row['华泰证券'],
            row['国泰君安'],
            row['tag'],
            row['AI分析结果'],
            row['风险等级(AI实现)'],
            row['手工调整意见'],
            row['edit_date']
        ))

    # 提交更改并关闭连接
    conn.commit()
    conn.close()
    logger.info('修改白名单数据已经入库,详见rating_db.edit_log')

query_config=init() #初始化参数



############################修改文件路径!!############################
file_path ='report/2025-03-26较同业券商偏离汇总报告.xlsx'
folder_path = 'report/'
######################################################################



key = input('修改file_path为最新了吗? 本日report运行生成了吗? 输入y/n')
if key == 'y':
    if os.path.exists(file_path):
        logger.info("文件存在")
    else:
        logger.info("本日较同业券商偏离汇总报告尚未生成,正在生成中... ")
        module_broker_tag_compare.broker_tag_compare(is_report=1)               ##需要生成报告改成1
else:
    logger.info("前往broker_tag_compare.py生成最新报告,并且修改file_path为最新")

2025-04-07 15:27:27,645 - my_logger - INFO - 前往broker_tag_compare.py生成最新报告,并且修改file_path为最新


#### 手工调整前调整edit_df

In [ ]:
############################################回忆上一次修改的edit_df#############################################
latest_date_query = '''
SELECT MAX(date_list.td_date) AS latest_date FROM
(SELECT DISTINCT q.td_date FROM shsz_stock_daily_quotation q
WHERE q.td_date<(
SELECT MIN(edit_date) FROM saas_five_class_white_list
)) date_list;
'''                               #####系统数据库是伦敦时区，需要+8小时
latest_date = pd.read_sql(latest_date_query,query_config['istock_conn'])
latest_date = latest_date.loc[0].dt.strftime('%Y-%m-%d').values[0]
true_date = input(f'当前最新修改记录日期是{latest_date},日期是否正确? (edit_date的n-1) 输入y/n:')
if true_date == 'y':
    pass
else:
    latest_date = input('请输入正确日期,修改日期(如若不知，请询问明非姐):')
########################################读入历史数据#############################################################
edit_df=pd.read_excel(file_path, sheet_name='异常票汇总', usecols='A:K',dtype={'stock_code': str, 'trade_date': str})
log_today_df=pd.read_excel(file_path, sheet_name='平滑后评级',dtype={'stock_code': str, 'trade_date': str})


sql_edit_whitesheet = '''SELECT sec_code AS stock_code ,stock_name,five_class_manual_adj FROM saas_five_class_white_list'''
whitesheet = pd.read_sql(sql_edit_whitesheet,query_config['istock_conn'])
# sql_broker_tag_sheet = f'''SELECT * FROM broker_tag_sheet WHERE trade_date = '{latest_date}';'''
# log_df = pd.read_sql(sql_broker_tag_sheet,query_config['rating_conn'])

log_df = module_broker_tag_compare.broker_compare_history(latest_date)

whitesheet =pd.merge(whitesheet,log_df,on=['stock_code','stock_name'],how='left')
###############################################################################################################

for col in edit_df.columns:
    if col not in log_today_df.columns:
        log_today_df[col] = float('nan')         ###对齐表结构

for stock_code in whitesheet['stock_code'].unique():
    now_rating_row = edit_df[edit_df['stock_code'] == stock_code]
    whitesheet_rating_row = whitesheet[whitesheet['stock_code'] == stock_code]
    if len(now_rating_row) == 0:
        if whitesheet_rating_row['SAAS_rating'].values[0] == whitesheet_rating_row['five_class_manual_adj'].values[0]:
            continue    ##上周认为不要调整的,现在不在tag中,不处理
        else:
            add_df = log_today_df.loc[log_today_df['stock_code']== stock_code]
            edit_df = pd.concat([edit_df,add_df])        ##上周要调整的,现在不在tag中,复核   
    else:
        if now_rating_row['SAAS_rating'].values[0] == whitesheet_rating_row['SAAS_rating'].values[0] and now_rating_row['中信建投'].values[0] == whitesheet_rating_row['中信建投'].values[0] and now_rating_row['华泰证券'].values[0] == whitesheet_rating_row['华泰证券'].values[0] and now_rating_row['国泰君安'].values[0] == whitesheet_rating_row['国泰君安'].values[0]:
            edit_df.loc[edit_df['stock_code'] == stock_code, '手工调整意见'] = whitesheet_rating_row['five_class_manual_adj'].values[0]    ####于上周一致的,默认不需要调整,人工复核
        else:
            continue
###############################################################################################################

attention_stock = pd.read_excel(r'password_attention\attention_list.xlsx',dtype={'stock_code': str})    ##"XXXXXX格式，不需要加同花顺后缀。"
attention_stock = attention_stock['股票代码'].tolist()
if len(attention_stock)==0:
    logger.info('无关注股票')
else:
    for stock_code in attention_stock:
        stock_code = str(stock_code)
        add_df = log_today_df.loc[log_today_df['stock_code']== stock_code]
        edit_df = pd.concat([edit_df,add_df])   
        edit_df.loc[edit_df['stock_code'] == stock_code, '手工调整意见'] = 'E'

edit_df['tag'].fillna(5, inplace=True)
edit_df['edit_date']=datetime.now().strftime('%Y-%m-%d')
logger.info('edit_df手动前调整完成!')

################################################################################################################
book = load_workbook(file_path)

# 定位到指定的工作表
sheet = book['异常票汇总']

# 清空A:K列的数据（即前11列）
for row in sheet.iter_rows(min_col=1, max_col=11, min_row=1, max_row=sheet.max_row):
    for cell in row:
        cell.value = None

# 保存修改
book.save(file_path)

# 使用ExcelWriter将新的DataFrame写入工作表
with pd.ExcelWriter(file_path, engine='openpyxl', mode='a', if_sheet_exists='overlay') as writer:
    # 注意这里不需要手动设置writer.book
    edit_df.to_excel(writer, sheet_name='异常票汇总', index=False, startrow=0, startcol=0)

logger.info('异常票汇总调整完成! 可以开始手工调整和ai评级!!')
logger.info(f'报告文件路径： {file_path} !!!')

2025-04-06 16:17:42,115 - my_logger - INFO - 2025-03-31是上一次更新时间!  (n-1)
2025-04-06 16:17:42,115 - my_logger - INFO - 以下是broker_tag_compare历史数据初始化!初始化日期不可信！！！ 开始取得上周历史修改数据!以2025-03-31为准!
2025-04-06 16:17:43,865 - my_logger - INFO - 初始化完成! 开始生成2025-04-03的同业对比报告
2025-04-06 16:17:50,314 - my_logger - INFO - 同业对比raw_data准备完毕!
2025-04-06 16:17:56,076 - my_logger - INFO - 整体数据集生成! 去重清洗完成!
2025-04-06 16:18:05,047 - my_logger - INFO - tag生成完成!
2025-04-06 16:18:05,048 - my_logger - INFO - 2025-03-31历史白名单数据已经回忆完成!
2025-04-06 16:18:05,203 - my_logger - INFO - 无关注股票
2025-04-06 16:18:05,204 - my_logger - INFO - edit_df手动前调整完成!
2025-04-06 16:19:00,422 - my_logger - INFO - 异常票汇总调整完成! 可以开始手工调整和ai评级!!
2025-04-06 16:19:00,423 - my_logger - INFO - 报告文件路径： report/2025-03-26较同业券商偏离汇总报告.xlsx !!!


#### AI分析prompt生成

In [ ]:
edit_df=pd.read_excel(file_path, sheet_name='异常票汇总', usecols='A:K',dtype={'stock_code': str, 'trade_date': str})
edit_df['tag'].fillna(5, inplace=True)
prompt_text = """你是一位股票投资顾问,请根据股票的基本面,技术面,行为面综合分析以下股票,评价该股票在未来短期内发生大幅下跌的概率,
每只股票75字左右,可以参考的因素包括：流通市值,每股收益增长率,营业利润增长率,经营活动现金流,净资产收益率,营业收入波动率,Beta,
短期反转,相对市场强度,Sigma,成交量,融资余额占总市值的比,分红水平,审计事务所变动频率,重大违规事件,机构持仓变动率,对外担保额度,
大股东减持,股权质押情况。生成成表格形式,表格形式分为四列：股票代码,股票名称,短期大幅下跌概率分析,风险评级,使我可以直接复制到excel,
请完整生成,不能仅仅是部分示例;股票如下:\n"""
# 提取并格式化输出
result = edit_df[['stock_code', 'stock_name']].apply(lambda row: '\t'.join(row), axis=1).tolist()
final_prompt = prompt_text + '\n'.join(result)
# 使用 StringIO 和 read_csv 模拟从文本创建 DataFrame 的过程
final_prompt = pd.read_csv(StringIO(final_prompt.replace("股票如下：\n", "")), 
                       sep='\t', header=None, names=['stock_code', 'stock_name'])

# 将生成的数据框内容复制到剪贴板
final_prompt.to_clipboard(index=False,header=False)
logger.info("AI_propmt已复制到剪贴板,可进行黏贴至cherrystudio进行分析,票过多请分段分析。")

#### saas_five_class_white_list 表更新（生产环境，务必在一天内完成准确的手动更新！）

In [ ]:
from sqlalchemy import create_engine, Table, Column, String, MetaData, insert
edit_df = pd.read_excel(file_path, sheet_name='异常票汇总', usecols='A:K', dtype={'stock_code': str, 'trade_date': str})
edit_df['edit_date'] = datetime.now().strftime('%Y-%m-%d')
update_df = edit_df[['stock_code', 'stock_name', '手工调整意见', 'edit_date']]
update_df.columns = ['sec_code', 'stock_name', 'five_class_manual_adj', 'edit_date']

confirm_str = input('输入 del and update 以确认修改：')
if confirm_str == 'del and update':
    save_edit_log(edit_df)  # edit_df记录进入rating_db

    record_df_all = pd.read_excel(file_path, sheet_name='平滑后评级', dtype={'stock_code': str, 'trade_date': str})
    module_query_update.insert_data_without_duplicates(record_df_all, 'broker_tag_sheet')

    engine = query_config['istock_conn']
    metadata = MetaData()

    # 定义目标表结构
    table = Table('saas_five_class_white_list', metadata,
                  Column('sec_code', String(50)),
                  Column('stock_name', String(50)),
                  Column('five_class_manual_adj', String(1)),
                  Column('edit_date', String(20)))

    # 准备插入的数据
    data_to_insert = [dict(row) for row in update_df.to_dict(orient='records')]
    try:
        with engine.begin() as connection:
            # 清空表
            delete_stmt = table.delete()
            connection.execute(delete_stmt)
            logger.info('清空旧白名单!!!')

            # 执行批量插入
            stmt = insert(table)
            connection.execute(stmt, data_to_insert)
            logger.info("！！！白名单更新完成！！！")
    except Exception as e:
        logger.info(f"发生错误: {e}")

    EmailAutomation(file_path,'' , "", edit_df)
